In [ ]:
!pip install qiskit scipy qiskit_aer matplotlib

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp

from qiskit_aer.primitives import Estimator
from qiskit_aer import AerSimulator

from qiskit.visualization import plot_histogram

from scipy.optimize import minimize
from scipy.constants import pi

In [ ]:
cost_matrix = [
  [ 0, 17, 30, 10,  6, 31],
  [17,  0, 10, 18, 41, 12],
  [30, 10,  0,  9, 20,  7],
  [10, 18,  9,  0, 12,  8],
  [ 6, 41, 20, 12,  0, 15],
  [31, 12,  7,  8, 15,  0],
]

steps = 5

In [ ]:
def cost_operator_subcircuit(n, cost_matrix, param):
    sub = QuantumCircuit(n, name="U_C")
    
    for i in range(n):
        for j in range(i + 1, n):
            sub.rzz(cost_matrix[i][j] * param, i, j)

    return sub.to_gate()

In [ ]:
def mixing_operator_subcircuit(n, param):
    sub = QuantumCircuit(n, name="U_M")
    sub.rx(param, range(n))

    return sub.to_gate()

In [ ]:
n = len(cost_matrix)
gamma = [Parameter(f"γ{i}") for i in range(steps)]
mu = [Parameter(f"μ{i}") for i in range(steps)]

qr = QuantumRegister(n, name="x")
cr = ClassicalRegister(n)
qc = QuantumCircuit(qr, cr)

qc.h(qr)

for i in range(steps):
    qc.append(cost_operator_subcircuit(n, cost_matrix, gamma[i]), qr)
    qc.append(mixing_operator_subcircuit(n, mu[i]), qr)

qc.measure(qr, cr)

qc.draw(fold=-1)

In [ ]:
max_cost = max(map(max, cost_matrix))

H_C = 0.5 * SparsePauliOp.from_list(
    [
        (
            "I" * (i) + "Z" + "I" * (j - i - 1) + "Z" + "I" * (5 - j),
            cost_matrix[i][j] / max_cost,
        )
        for i in range(n)
        for j in range(i + 1, n)
    ]
)

In [ ]:
estimator = Estimator()

def cost(params):
    val = estimator.run(qc, H_C, parameter_values=params).result().values[0]
    return val

In [ ]:
initial_params = [pi] * steps + [pi / 2] * steps
optimum = minimize(cost, x0=initial_params, method="Powell")

optimal_params = optimum.x
optimal_qc = qc.assign_parameters(optimal_params)

In [ ]:
sim = AerSimulator()
result = sim.run(optimal_qc.decompose(), shots=1_000).result()
counts = result.get_counts()

In [ ]:
plot_histogram(counts)